Import basic libraries

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from scipy.stats import zscore
import random
import pymysql

Fetch data from SQL server to Python

In [6]:
host = "localhost"
port= 3306
database = 'burger_sales'
user = 'root'
password = 'ranjeetjb'

conn = pymysql.connect(host=host,
                      user=user,
                      password=password,
                      database=database,
                      port=port)
query = "SELECT * FROM burger_sales_data;"
df=pd.read_sql(query, conn)
conn.close()
print(df.head())

C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_15528\3162228948.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query, conn)


         Date Region  Temperature  Humidity       Wind  Visibility  \
0  15-09-2020   Reg1    10.248814  0.779164  11.509130   14.503403   
1  14-09-2020   Reg1    10.337595  0.908549   7.432656    2.232960   
2  13-09-2020   Reg1    20.763686  0.505324   7.788249    4.779211   
3  12-09-2020   Reg1    21.500892  0.758557   3.767432    9.904534   
4  11-09-2020   Reg1    21.774269  0.398296  20.705369   15.224605   

      Pressure    Sales  
0  1017.293917   991.60  
1  1019.452636  1858.59  
2  1022.677119     3.99  
3  1009.341357  3090.78  
4  1015.713234   990.99  


Data preprocessing

In [7]:
# Data Preprocessing
def preprocess_data(df):
    """Handle missing values, outliers, and feature engineering."""
    # 1. Handle Missing Values
    imputer = SimpleImputer(strategy='median')
    df[['Temperature', 'Humidity', 'Wind', 'Visibility', 'Pressure', 'Sales']] = imputer.fit_transform(df[['Temperature', 'Humidity', 'Wind', 'Visibility', 'Pressure', 'Sales']])

    # 2. Handle Outliers using Z-score method
    z_scores = np.abs(zscore(df[['Temperature', 'Humidity', 'Wind', 'Visibility', 'Pressure', 'Sales']]))
    threshold = 3
    df_no_outliers = df[(z_scores < threshold).all(axis=1)]

    # 3. Feature Engineering: Lag Features and Rolling Mean
    df_no_outliers['Sales_lag_1'] = df_no_outliers['Sales'].shift(1)
    df_no_outliers['Sales_lag_7'] = df_no_outliers['Sales'].shift(7)
    df_no_outliers['Sales_rolling_mean_7'] = df_no_outliers['Sales'].rolling(window=7).mean()
    df_no_outliers.dropna(inplace=True)  # Drop NaN values created by lag features and rolling mean

    return df_no_outliers

# Prepare Data for LSTM
def prepare_data(df, window_size=30):
    """Prepare time series data for LSTM."""
    sales = df['Sales'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))  # Scale data to range (0, 1)
    scaled_sales = scaler.fit_transform(sales)

    X, y = [], []
    for i in range(window_size, len(scaled_sales)):
        X.append(scaled_sales[i-window_size:i, 0])
        y.append(scaled_sales[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # 3D shape for LSTM (samples, timesteps, features)

    return X, y, scaler

Build LSTM model

In [8]:
# Function to build LSTM model with hyperparameters
def build_model(units1, units2, dropout_rate, learning_rate):
    """Build and compile the LSTM model."""
    model = Sequential()
    model.add(LSTM(units=units1, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))  # Single output node for regression
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])
    return model

# Random Search for Hyperparameters
def random_search_hyperparameters(X_train, y_train, X_val, y_val, n_iter=20):
    """Randomly search for the best hyperparameters for LSTM."""
    best_score = float('inf')
    best_params = None
    best_model = None

    for i in range(n_iter):
        print(f"Random Search Iteration: {i+1}/{n_iter}")

        # Randomly select hyperparameters
        units1 = random.choice([32, 64, 128, 256])
        units2 = random.choice([32, 64, 128, 256])
        dropout_rate = random.uniform(0.1, 0.5)
        learning_rate = 10 ** random.uniform(-4, -2)

        print(f"Testing Hyperparameters: units1={units1}, units2={units2}, dropout_rate={dropout_rate:.4f}, learning_rate={learning_rate:.6f}")

        # Build and train the model
        model = build_model(units1, units2, dropout_rate, learning_rate)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                            validation_data=(X_val, y_val),
                            callbacks=[early_stopping],
                            verbose=1)

        val_loss = min(history.history['val_loss'])
        print(f"Validation Loss: {val_loss}")

        if val_loss < best_score:
            best_score = val_loss
            best_params = {'units1': units1, 'units2': units2, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate}
            best_model = model

    return best_model, best_params, best_score

In [9]:
df = preprocess_data(df)
df.head()

C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_15528\2885962469.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers['Sales_lag_1'] = df_no_outliers['Sales'].shift(1)
C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_15528\2885962469.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers['Sales_lag_7'] = df_no_outliers['Sales'].shift(7)
C:\Users\Ranjeet Kumar\AppData\Local\Temp\ipykernel_15528\2885962469.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy

,Date,Region,Temperature,Humidity,Wind,Visibility,Pressure,Sales,Sales_lag_1,Sales_lag_7,Sales_rolling_mean_7
7,08-09-2020,Reg1,25.493278,0.428529,18.176798,14.406207,1014.294212,1584.29,3353.17,991.60,1559.105714
8,07-09-2020,Reg1,23.472156,0.464102,8.725919,4.130202,1020.738999,107.60,1584.29,1858.59,1308.964286
9,06-09-2020,Reg1,24.451157,0.686122,6.219998,10.344838,1010.946550,3108.91,107.60,3.99,1752.524286
10,05-09-2020,Reg1,25.494625,0.401804,19.308957,14.123511,1016.493160,960.01,3108.91,3090.78,1448.128571
11,04-09-2020,Reg1,25.673838,0.500355,10.796126,6.519813,1018.567744,415.47,960.01,990.99,1365.911429


Splitting of data

In [10]:
# Prepare the data for LSTM
X, y, scaler = prepare_data(df, window_size=30)

# Split data into train and validation sets
train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]


Evaluation of model

In [ ]:
# Hyperparameter search
best_model, best_params, best_score = random_search_hyperparameters(X_train, y_train, X_val, y_val, n_iter=20)
print(f"Best Hyperparameters: {best_params}")
print(f"Best Validation Loss: {best_score}")

# Evaluate the model
y_pred = best_model.predict(X_val)

**Metrics** calculated

In [11]:
# Calculate evaluation metrics
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Best Hyperparameters: {best_params}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-Squared (R2): {r2}")

# Plot actual vs predicted sales
plt.figure(figsize=(12, 6))
plt.plot(y_val, label='Actual Sales')
plt.plot(y_pred, label='Predicted Sales')
plt.legend()
plt.title('Actual vs Predicted Sales')
plt.show()

NameError: name 'y_pred' is not defined

Load model to Keras

In [12]:
from tensorflow.keras.models import load_model

loaded_model = load_model('sales_forecasting_lstm_model.keras')

C:\Users\Ranjeet Kumar\anaconda3\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Validation

In [13]:
# Assume X_val is the validation data you want to predict on
predictions = loaded_model.predict(X_val)

# Print first 5 predictions
print(predictions[:5])

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
[[0.30338556]
 [0.6905669 ]
 [0.1681576 ]
 [0.70780647]
 [0.23367608]]


Evaluation on validation set

In [14]:
# Evaluate the model on the validation set
loss, mae = loaded_model.evaluate(X_val, y_val)
print(f"Loss: {loss}, Mean Absolute Error (MAE): {mae}")

150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0626 - mae: 0.2049
Loss: 0.06079982593655586, Mean Absolute Error (MAE): 0.20116250216960907


Actual sales

In [15]:
import numpy as np

# Example predictions (replace this with your predictions)
scaled_predictions = np.array([[0.30338556], [0.6905669], [0.1681576], [0.70780647], [0.23367608]])

# Inverse transform the predictions
predicted_sales = scaler.inverse_transform(scaled_predictions)

# Print results
print("Predicted Sales (Actual Scale):", predicted_sales)


Predicted Sales (Actual Scale): [[1081.54142235]
 [2461.60284702]
 [ 599.53758629]
 [2523.05122554]
 [ 833.07034603]]


Burger forecasting sales for 5 days

In [16]:
# Extract recent 'Sales' data only (same as training)
window_size = 5


latest_sales = df['Sales'].values[-window_size:].reshape(-1, 1)

# Scale using the original 'Sales' scaler
scaled_latest_sales = scaler.transform(latest_sales)

# Reshape to match LSTM input (1, 30, 1)
X_input = np.reshape(scaled_latest_sales, (1, window_size, 1))

# Make prediction
next_day_prediction = loaded_model.predict(X_input)

# Inverse transform the prediction
actual_prediction = scaler.inverse_transform(next_day_prediction)

print("Forecasted Sales for Next Day:", actual_prediction[0][0])



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
Forecasted Sales for Next Day: 690.4345
